# Testes de Modelos, Rag, Etc

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import google.generativeai as genai
import json

models = genai.list_models()
models = [m.__dict__ for m in models]
models
with open("google_list_models.json", "w") as f:
    json.dump(models, f, indent=4)
[m["name"] for m in models]

['models/chat-bison-001',
 'models/text-bison-001',
 'models/embedding-gecko-001',
 'models/gemini-1.0-pro-vision-latest',
 'models/gemini-pro-vision',
 'models/gemini-1.5-pro-latest',
 'models/gemini-1.5-pro-001',
 'models/gemini-1.5-pro-002',
 'models/gemini-1.5-pro',
 'models/gemini-1.5-flash-latest',
 'models/gemini-1.5-flash-001',
 'models/gemini-1.5-flash-001-tuning',
 'models/gemini-1.5-flash',
 'models/gemini-1.5-flash-002',
 'models/gemini-1.5-flash-8b',
 'models/gemini-1.5-flash-8b-001',
 'models/gemini-1.5-flash-8b-latest',
 'models/gemini-1.5-flash-8b-exp-0827',
 'models/gemini-1.5-flash-8b-exp-0924',
 'models/gemini-2.0-flash-exp',
 'models/gemini-2.0-flash',
 'models/gemini-2.0-flash-001',
 'models/gemini-2.0-flash-exp-image-generation',
 'models/gemini-2.0-flash-lite-001',
 'models/gemini-2.0-flash-lite',
 'models/gemini-2.0-flash-lite-preview-02-05',
 'models/gemini-2.0-flash-lite-preview',
 'models/gemini-2.0-pro-exp',
 'models/gemini-2.0-pro-exp-02-05',
 'models/gemin

In [10]:
from llms.models import load_model
llm, model_name, max_new_tokens = load_model("gemini-1.5-flash-8b")

load_model...
model_alias='gemini-1.5-flash-8b' max_new_tokens=8192 model_data=('google', 'gemini-1.5-flash-8b')
Loading ChatGoogleGenerativeAI model...
llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash-8b', google_api_key=SecretStr('**********'), temperature=0.0, max_output_tokens=8192, max_retries=3, timeout=60.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002AC4A99A350>, default_metadata=())
llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash-8b', google_api_key=SecretStr('**********'), temperature=0.0, max_output_tokens=8192, max_retries=3, timeout=60.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002AC4AA345D0>, default_metadata=())
llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash-8b', google_api_key=SecretStr('**********'), temperature=0.0, max_output_tokens=8192, max_retries=3, timeout=60.0, client=<google.ai.

In [12]:
llm.switch_models()

Current model: 0 models/gemini-1.5-flash-8b XWPQ
Next model:    1 models/gemini-1.5-flash-8b PgdM


In [13]:
from llms.models import query_model
query_model(llm, "O que é a vida")

query_model_google...
len(prompt)=14 model_name='models/gemini-1.5-flash-8b' key_index=1 key[-4:]='PgdM'
len(response)=2159
response='A vida é um conceito complexo e multifacetado, sem uma definição única e universalmente aceita.  Depende muito do contexto em que é considerada.  Podemos abordar a questão de diferentes perspectivas:\n\n**Do ponto de vista biológico:** A vida é geralmente definida como um sistema complexo que apresenta as seguintes características:\n\n* **Organização:**  Estrutura celular, com componentes interdependentes.\n* **Metabolismo:** Capacidade de obter e utilizar energia para realizar funções vitais.\n* **Reprodução:** Capacidade de gerar novos indivíduos.\n* **Adaptação:** Capacidade de responder a mudanças no ambiente.\n* **Homeostase:** Manutenção de um ambiente interno estável.\n* **Hereditariedade:** Transmissão de características genéticas para a prole.\n\n**Do ponto de vista filosófico:** A vida pode ser vista como:\n\n* **Experiência consciente:** A cap

('A vida é um conceito complexo e multifacetado, sem uma definição única e universalmente aceita.  Depende muito do contexto em que é considerada.  Podemos abordar a questão de diferentes perspectivas:\n\n**Do ponto de vista biológico:** A vida é geralmente definida como um sistema complexo que apresenta as seguintes características:\n\n* **Organização:**  Estrutura celular, com componentes interdependentes.\n* **Metabolismo:** Capacidade de obter e utilizar energia para realizar funções vitais.\n* **Reprodução:** Capacidade de gerar novos indivíduos.\n* **Adaptação:** Capacidade de responder a mudanças no ambiente.\n* **Homeostase:** Manutenção de um ambiente interno estável.\n* **Hereditariedade:** Transmissão de características genéticas para a prole.\n\n**Do ponto de vista filosófico:** A vida pode ser vista como:\n\n* **Experiência consciente:** A capacidade de sentir, pensar, perceber e ter consciência de si mesmo.\n* **Significado e propósito:** A busca por um sentido na existên

In [31]:
from llms.rag import load_rag
vector_store, rag_alias, embeddings_name = load_rag("google-ip")

load_rag...
load_embedding...
model_alias='google-4' model_data=('google', 'models/text-embedding-004') task_type='retrieval_query'
Loading GoogleGenerativeAIEmbeddings model...
embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002425765BF50>, model='models/text-embedding-004', task_type='retrieval_query', google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)
rag_alias='google-ip' embedding_model_alias='google-4' embeddings_name='models/text-embedding-004' path_index='data/faiss_index_google_v4_ip'
Loading index...


  4%|▍         | 1/23 [00:00<00:03,  7.10it/s]

i=0 vector_store.index.ntotal=10000
i=1 vector_store.index.ntotal=20000

 13%|█▎        | 3/23 [00:00<00:03,  5.75it/s]


i=2 vector_store.index.ntotal=30000


...

i=17 vector_store.index.ntotal=172923
i=18 vector_store.index.ntotal=182923


 91%|█████████▏| 21/23 [00:04<00:00,  4.31it/s]

i=19 vector_store.index.ntotal=192923
i=20 vector_store.index.ntotal=202923


100%|██████████| 23/23 [00:05<00:00,  4.40it/s]

i=21 vector_store.index.ntotal=212923
i=22 vector_store.index.ntotal=222923
Index loaded.
load_index took: 5.24207067489624


In [ ]:
from llms.models import load_embedding
embeddings, model_name = load_embedding("gemini")
emb = embeddings.embed_query("hello!!")
emb

load_embedding...
model_alias='gemini' model_data=('google', 'models/gemini-embedding-exp-03-07') task_type='retrieval_query'
Loading GoogleGenerativeAIEmbeddings model...
embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000024257649890>, model='models/gemini-embedding-exp-03-07', task_type='retrieval_query', google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)


In [39]:
len(emb)

3072

In [ ]:
# Verificando que a norma dos modelos de embedding utilizados já é 1
import numpy as np
np.linalg.norm(emb)

1.0000000125368993

In [ ]:
from llms.rag import query_index
docs, context = query_index(
    vector_store=vector_store,
    query="Qual hotel tem o melhor café?",
    filter={"nome": {"$eq": "Ancona Center Hotel"}, "nota_avaliacao": {"$gte": 3}},
    n_responses=10,
)
context

Querying index...
Query with increasing fetch_k: i=0 fetch_k=1000
len(results)=6/fetch_k=1000 n_responses=10
Query with increasing fetch_k: i=1 fetch_k=10000
len(results)=35/fetch_k=10000 n_responses=10
Results sufficient.
 - Avaliação 1, Similaridade: 0.686
Hotel: Ancona Center Hotel, 3 Estrelas.
Região:SUDESTE; Estado:SÃO PAULO; Cidade:Amparo
Tipo:Hotel; Classificação:4.2; Quantidade Avaliações:717
Nota:3; Curtidas:0; Usuário é guia local; Nota Quartos:3; Nota Localização:5; Nota Serviço:4
Avaliação: Bom hotel, bem localizado, acho que precisa renovar os baheiros, apenas. Café da manhã completo, muito bem servido. Atendimento simpático.


 - Avaliação 2, Similaridade: 0.687
Hotel: Ancona Center Hotel, 3 Estrelas.
Região:SUDESTE; Estado:SÃO PAULO; Cidade:Amparo
Tipo:Hotel; Classificação:4.2; Quantidade Avaliações:717
Nota:5; Curtidas:1; Usuário é guia local
Avaliação: Atendimento  maravilhoso, limpeza,café da manhã  diversificado e  muito  gostoso. Recomendo.


 - Avaliação 3, Similar

' - Avaliação 1, Similaridade: 0.686\nHotel: Ancona Center Hotel, 3 Estrelas.\nRegião:SUDESTE; Estado:SÃO PAULO; Cidade:Amparo\nTipo:Hotel; Classificação:4.2; Quantidade Avaliações:717\nNota:3; Curtidas:0; Usuário é guia local; Nota Quartos:3; Nota Localização:5; Nota Serviço:4\nAvaliação: Bom hotel, bem localizado, acho que precisa renovar os baheiros, apenas. Café da manhã completo, muito bem servido. Atendimento simpático.\n\n - Avaliação 2, Similaridade: 0.687\nHotel: Ancona Center Hotel, 3 Estrelas.\nRegião:SUDESTE; Estado:SÃO PAULO; Cidade:Amparo\nTipo:Hotel; Classificação:4.2; Quantidade Avaliações:717\nNota:5; Curtidas:1; Usuário é guia local\nAvaliação: Atendimento  maravilhoso, limpeza,café da manhã  diversificado e  muito  gostoso. Recomendo.\n\n - Avaliação 3, Similaridade: 0.687\nHotel: Ancona Center Hotel, 3 Estrelas.\nRegião:SUDESTE; Estado:SÃO PAULO; Cidade:Amparo\nTipo:Hotel; Classificação:4.2; Quantidade Avaliações:717\nNota:3; Curtidas:0; Usuário é guia local\nAvalia

In [22]:
# Test all models
# for a in models_text:
#     if models_text[a][0] in ["local","google"]:
#         continue
#     print(a)
#     llm, model_name, max_new_tokens = load_model(a)
#     llmresult = llm.generate(["hi"])
#     print(llmresult)
#     response = llmresult.generations[0][0].text
#     print(response)